<a href="https://colab.research.google.com/github/casalazara/Machine-Learning-IELE/blob/master/Proyecto/Proyecto_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

# Carga de datos

## Directorio con los datos

In [2]:
drive.mount('/gdrive')
path = "../gdrive/MyDrive/Data"

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Separación de los datos

In [ ]:
data = []
labels = []
genres = ["Cumbia","Vallenato","Salsa","Merengue","Bachata"]
for i in range(len(genres)):   
    genre=genres[i]
    for img in os.listdir(path+"/"+genre):
      image=tf.keras.preprocessing.image.load_img(path+"/"+genre+"/"+img, color_mode='rgb', target_size= (300,300))
      image=np.array(image)
      data.append(image)
      labels.append(i)

data = np.array(data)
labels = np.array(labels)
labels = to_categorical(labels, 5)
X_train, X_test_val, y_train, y_test_val = train_test_split(data, labels, test_size=0.2, random_state=42)
X_test,X_val,y_test,y_val = train_test_split(X_test_val,y_test_val,test_size=.5,random_state=42)
X_train.shape, X_test.shape, X_val.shape

In [ ]:
X_train[:2]

In [ ]:
y_train[:2]

In [ ]:
# Create an ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.)
validation_datagen = ImageDataGenerator(rescale = 1./255.)
train_generator = train_datagen.flow(X_train, y_train, batch_size=8)
validation_generator = validation_datagen.flow(X_val, y_val, batch_size=8)

# Modelo


## Carga del modelo

In [ ]:
model=VGG16(include_top=False,input_shape=(300,300,3))

#Congelamiento de capas

In [ ]:
for layer in model.layers:
    layer.trainable=False

## Últimas capas

In [ ]:
output=model.layers[-1].output
model_final=tf.keras.layers.Flatten()(output)
model_final=tf.keras.layers.Dense(512,activation='relu')(model_final)
model_final=tf.keras.layers.Dense(64,activation='relu')(model_final)
model_final=tf.keras.layers.Dense(5,activation='softmax')(model_final)

## Modelo final

In [ ]:
model=tf.keras.models.Model(model.input,model_final)

## Callback

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def __init__(self, weights_dir, patience=10, rate=0.5):
        """
        Initializer Method
        :param weights_dir: The directory of the model weights
        :param patience: Number of epochs before reducing lr
        :param rate: The rate to reduce the lr
        """
        super(CustomCallback, self).__init__()
        self.weights_dir = weights_dir
        self.rate = rate
        self.patience = patience

    def on_train_begin(self, logs=None):
        """
        Configurations to be taken an account on
        before training begins.
        :param logs: The logs from the output.
        """
        self.best_loss = np.Inf
        self.best_recall = 0
        self.wait = 0
        self.best_weights = None
    
    def on_epoch_end(self, epoch, logs=None):
        """
        Method that is called each time an epoch ends
        excecution
        :param epoch: The current epoch of the training
        :param logs: The logs from the output from an epoch ended
        """
        # Current validation loss
        current_loss = logs.get("val_loss")
        # Current validation Recall
        current_recall = logs.get("val_Recall")
        if (current_loss < self.best_loss or 
            current_recall > self.best_recall):
            # We save the model weights
            # We wait again
            # We assign the best loss and the best recall.
            self.wait = 0
            self.best_loss = current_loss
            self.best_recall = current_recall
            self.model.save_weights(self.weights_dir)
            self.best_weights = self.model.get_weights()
            print("\nBest Weights Saved!!")
        else:
            self.wait += 1
            if self.wait >= self.patience:
                # Reduce learning rate by the initialized rate.
                self.wait = 0
                print(f"\nEpoch {epoch}, Reducing Learning Rate")
                # We set the new learning rate
                lr = K.get_value(self.model.optimizer.lr)
                new_lr = lr * self.rate
                K.set_value(self.model.optimizer.lr, new_lr)
                print(f"\nLearning Rate Reduced: {new_lr}")
                # We load the best weights to continue the training.
                self.model.set_weights(self.best_weights)
                print("\nBest Weights Loaded!!")


## Compilación

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc','Recall'])

## Entrenamiento

In [ ]:
history = model.fit_generator(train_generator, epochs=2,validation_data = validation_generator, callbacks = [CustomCallback('./train_best_weights.h5')])

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {test_loss}, Test acc: {test_acc}')

## Evaluación

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Guardado del modelo y pesos

In [ ]:
model.save_weights('best_weights.h5')
model.save('model.h5')